# Masking Demonstration

In [5]:
import torch

In [6]:
test = torch.tensor([[1,2,3,4],[1,2,3,4],[1,2,3,4],[1,2,3,4]])
test

tensor([[1, 2, 3, 4],
        [1, 2, 3, 4],
        [1, 2, 3, 4],
        [1, 2, 3, 4]])

In [7]:
new_test = test
new_test

tensor([[1, 2, 3, 4],
        [1, 2, 3, 4],
        [1, 2, 3, 4],
        [1, 2, 3, 4]])

In [8]:
masked = torch.rand((4,1)) < 0.75 # we use mask_prop of 0.75 to showcase majority masked
masked.type('torch.DoubleTensor')

tensor([[0.],
        [1.],
        [0.],
        [1.]], dtype=torch.float64)

In [9]:
masked = masked.type_as(new_test)

new_test[:, torch.tensor([2,3])] *= (1-masked)
new_test

tensor([[1, 2, 3, 4],
        [1, 2, 0, 0],
        [1, 2, 3, 4],
        [1, 2, 0, 0]])

Note that in this case, in column indexes 2 and 3 (our sample response genes for the exercise) we get 0s in the row indexes where the row value in _masked_ is 1. In this case in particular, row indexes 0, 1, and 2 were randomly selected to be masked, so rows 0, 1, and 2 have values of 0 for the response genes.

In [10]:
test = torch.rand(100,100)

In [16]:
masked = torch.rand(1,100) < 0.5

In [17]:
masked = masked.type_as(test)

In [18]:
newtest = test * (1-masked)

In [19]:
test

tensor([[0.4574, 0.8422, 0.7351,  ..., 0.0088, 0.0880, 0.1698],
        [0.3491, 0.7894, 0.5229,  ..., 0.0920, 0.7091, 0.0861],
        [0.4738, 0.2191, 0.8153,  ..., 0.9786, 0.5021, 0.3793],
        ...,
        [0.7611, 0.3610, 0.3346,  ..., 0.0709, 0.4123, 0.9567],
        [0.5006, 0.6794, 0.2416,  ..., 0.1410, 0.6593, 0.5007],
        [0.3832, 0.1256, 0.2213,  ..., 0.6657, 0.1721, 0.1937]])

In [20]:
newtest

tensor([[0.0000, 0.8422, 0.0000,  ..., 0.0088, 0.0000, 0.1698],
        [0.0000, 0.7894, 0.0000,  ..., 0.0920, 0.0000, 0.0861],
        [0.0000, 0.2191, 0.0000,  ..., 0.9786, 0.0000, 0.3793],
        ...,
        [0.0000, 0.3610, 0.0000,  ..., 0.0709, 0.0000, 0.9567],
        [0.0000, 0.6794, 0.0000,  ..., 0.1410, 0.0000, 0.5007],
        [0.0000, 0.1256, 0.0000,  ..., 0.6657, 0.0000, 0.1937]])